# Setup VastDB using Python API

**WARNING**: These example is untested.

First install the vastpy library

In [ ]:
!pip install vastpy

Next create a function that uses the Vast VMS API to configure a Vast Database.

In [ ]:
from vastpy import VASTClient

def base_setup(address, username, password, config):
    client = VASTClient(user=username, password=password, address=address)

    database_owner = config['user']['database_owner']

    try:
        client.vippools.post(**config['vippool'])
        print(f"VIP pool created: {config['vippool']}")
    except Exception as e:
        print(e)
        raise e

    try:
        client.users.post(**config['user'])
        print(f"User created: {config['user']}")
    except Exception as e:
        print(e)
        raise e

    user = client.users.get(name=database_owner)[0]
    user_id = user['id']

    try:
        response = client.users[user_id].access_keys.post(id=user_id)
        access_key = response['access_key']
        secret_key = response['secret_key']
        print(f"Key Generated for {database_owner}")
    except Exception as e:
        print(e)
        raise e

    try:
        response = client.views.post(
            path=config['viewpath']['path'],
            protocols=config['viewpath']['protocols'],
            create_dir=config['viewpath']['create_dir'],
            bucket=config['viewpath']['bucket'],
            bucket_owner=config['viewpath']['bucket_owner'],
            policy_id=config['viewpath']['policy_id']
        )
        print(f"View Created: {config['viewpath']['path']}")
    except Exception as e:
        print(e)
        raise e

    try:
        response = client.schemas.post(
            name=config['database']['schema'],
            database_name=config['database']['name']
        )
        print("Schema Created")
    except Exception as e:
        print(e)
        raise e

    print(access_key, secret_key)

Ensure the config below matches your environment and requirements

In [ ]:
config = {
    'vippool': {
        'start_ip': '11.0.0.2',
        'end_ip': '11.0.0.3',
        'subnet_cidr': 24
    },
    'user': {
        'local': True,
        'name': 'admin',
        'uid': 555,
        'allow_create_bucket': True,
        's3_superuser': True
    },
    'viewpath': {
        'path': '/demo-path',
        'protocols': ['S3', 'DATABASE'],
        'create_dir': True,
        'bucket': '/demo-bucket',
        'bucket_owner': 'demo-owner',
        'policy_id': 3 
    },
    'database': {
        'name': 'demo_db',
        'schema': 'demo_schema'
    }
}

Now call base_setup() with the config details.

In [ ]:

base_setup(
    address='http://11.0.0.2:9090',
    username='admin',
    password='password',
    config=config
    )